## Let's make two different integrators!

The first integrator will drift toward a positive boundary; the second will drift toward a negative boundary. 

In [ ]:
num_trials = 30;

# Instate the DDM mechanism
pos_integrator = pnl.DDM(function = pnl.DriftDiffusionIntegrator(
                              noise=0.5,
                              initializer=0,
                              starting_point=0,
                              rate=3.0
                            )
                         )


# Tell the DDM to store its outputs at each trial
pos_integrator.log.set_log_conditions(items=[pnl.DECISION_VARIABLE, pnl.RESPONSE_TIME])

# Put the DDM into a composition
DDM_positive = pnl.Composition(name = 'DDM_positive')
DDM_positive.add_node(node = pos_integrator)

# Run it
DDM_positive.run(inputs={pos_integrator: np.random.normal(1,.5,(1,1,1))},
num_trials=num_trials,
context='execid')

# # Extract the logged data
log = pos_integrator.log.nparray_dictionary()['execid']
decision_pos = log[pnl.DECISION_VARIABLE]
time_pos = log[pnl.RESPONSE_TIME]

# Plot!
plt.plot(decision_pos[:,0],'.--')
plt.title('Decision Progress Versus Time')
plt.ylabel('Decision Variable')
plt.xlabel('Time')
plt.xticks(np.arange(0,num_trials+1,int(num_trials/5)))
plt.show()

In [ ]:
num_trials = 30;

# Instate the DDM mechanism
neg_integrator = pnl.DDM(function = pnl.DriftDiffusionIntegrator(
                              noise=0.5,
                              initializer=0,
                              starting_point=0,
                              rate=3.0
                            )
                         )

# Tell the DDM to store its outputs at each trial
neg_integrator.log.set_log_conditions(items=[pnl.DECISION_VARIABLE, pnl.RESPONSE_TIME])

# Put the DDM into a composition
DDM_neg = pnl.Composition(name = 'DDM_negative')
DDM_neg.add_node(node = neg_integrator)

# Run it
DDM_neg.run(inputs={neg_integrator: -np.random.normal(1,.5,(1,1,1))},
num_trials=num_trials,
context='execid')

# Extract the logged data
log = neg_integrator.log.nparray_dictionary()['execid']
decision_neg = log[pnl.DECISION_VARIABLE]
time_neg = log[pnl.RESPONSE_TIME]

# Plot!
plt.plot(decision_neg[:,0],'.--')
plt.title('Decision Progress Versus Time')
plt.ylabel('Decision Variable')
plt.xlabel('Time')
plt.xticks(np.arange(0,num_trials+1,int(num_trials/5)))
plt.show()

### <a id='e6'>Exercise 4: Understanding PNL DDMs</a> 

1. The integrator attribute .output_values returns two values. What do they mean? 
2. Explain the difference in outputs for the two accumulators.
3. How many time steps does it take for the positive integrator to meet a threshold of .5? What about a threshold of 10?
4. How could you make the positive integrator reach threshold faster? What would you do to each of the following: 
    4a. Threshold?
    4b. Drift rate?
    4c. Starting point?
    4d. Noise?

### <a id='e7'>Exercise 5: Integrate Until Threshold</a> 

The following cell shows you how to run an accumulator until it reaches threshold using PsyNeuLink.

Rewrite this code so that it will take the pos_integrator and run until it hits threshold. Have your function return the time point at which your function hits threshold and the position of the decision at each time point. Plot the trajectory of the decision.

In [ ]:
D = pnl.DDM(function = pnl.DriftDiffusionIntegrator(threshold=[20.0]))
positive_moving_stimulus = np.array([[[1]], [[1.5]], [[2]], [[1]], [[1.5]]])

D.log.set_log_conditions(items=[pnl.DECISION_VARIABLE, pnl.RESPONSE_TIME])

C = pnl.Composition(name = 'DDM')
C.add_node(node = D)

C.termination_processing = {pnl.TimeScale.TRIAL: pnl.WhenFinished(D)}
C.run(inputs={D:positive_moving_stimulus}, context='execid') 

log = D.log.nparray_dictionary()['execid']
decisions = log[pnl.DECISION_VARIABLE]
times = log[pnl.RESPONSE_TIME]

### <a id='e8'>Exercise 6: Analytic Solutions to the DDM</a> 

Review Bogacz et al (2006) to learn about analytic solutions to the DDM. 

1. Construct a Bogacz integrator using the same parameters of the pos_integrator. (Hint: this should look something like: pnl.DDM(name="example_ddm",function=pnl.DriftDiffusionAnalytical(drift_rate=?,starting_point=?, threshold=?, noise=?, t0=?))
2. Run your Bogacz integrator with 5 different kinds of input, once each; use these 5 values as the input: [[.5], [1], [1.5], [2], [2.5]].  Acquire the statistics for the distributions of RTs for these values using an output states dictionary and the additional output states available to the Bogacz DDM.
3. Now run the pos_integrator using integrate to threshold termination conditions.   
4. Explain the pattern of RTs you observe.

In [ ]:
# This block of code handles parts 1 and 2

D = pnl.DDM(function = pnl.DriftDiffusionAnalytical(threshold=[20.0],
                                                   drift_rate = 3.0,
                                                   noise = 0.5),
           output_ports = [pnl.RESPONSE_TIME,pnl.PROBABILITY_UPPER_THRESHOLD,pnl.PROBABILITY_LOWER_THRESHOLD,pnl.RT_CORRECT_MEAN	])
positive_stimulus = np.array([[[1.7]]])
#[[1.5]], [[2]], [[1]], [[1.5]]])

D.log.set_log_conditions(items=[pnl.RESPONSE_TIME])  

C = pnl.Composition(name = 'DDM')
C.add_node(node = D)
outs = C.run(inputs={D:positive_stimulus},
              num_trials = 1, 
             context='execid') 

log = D.log.nparray_dictionary()['execid']
times = log[pnl.RESPONSE_TIME]


In [ ]:
# This block of code handles parts 3 onward. Notice that we put the ENTIRE 
# declaration of the composition into a loop. If we don't do this, we will get 
# the same results every time. In other words, our results will be non-random.
n_t = 10
for i in range(1,n_t):
  
  D = pnl.DDM(function = pnl.DriftDiffusionIntegrator(threshold=[20.0],
                                                   rate = 0.0,
                                                   noise = 1.0))

  positive_stimulus = np.array([[[.7]]])


  C = pnl.Composition(name = 'DDM')
  C.add_node(node = D)

  C.termination_processing = {pnl.TimeScale.TRIAL: pnl.WhenFinished(D)}


  outs = C.run(inputs={D:positive_stimulus}
                       ,context='execid') 
  print(outs)


### <a id='l0'>Exercise 7: Applying the DDM to Data</a> 


How do the parameters of the DDM map on to reaction times in lab experiments and real life decision-making?

We have some hypothetical experimental data from different experimental conditions for you to analyze. In this task, subjects had to respond to the color or direction of moving stimuli. A cue would preceed each trial to indicate whether the trial was a color trial or direction trial.

Conditions:

A.

1. In condition A1, subjects get no penalties for wrong answers.
2. In condition A2, subjects get a penalty for responding incorrectly.

B. 

3. In condition B1, subjects saw stimuli with high coherence, so all the stimuli were one color, or they were all moving in the same direction.
4. In condition B2, subjects saw stimuli with low coherence, so the stimuli were composed of multiple colors, or they saw stimuli moving in multiple directions.


Q1. Build a single path integrator (not analytic) that corresponds to one of the response options for A1.  Then build the analogue for A2, differing by only one parameter.   Do the same for B1 and B2 (the parameter distinguishing B1 and B2 should be different than for A1 and A2).  (When you are done you will have 4 integrators.) 

Q2. Run 100 iterations of each integrator until it meets threshold, and plot the resulting RT distributions. 

Q3. How do your RT distributions compare to the actual data (plotted for you below)?

In [1]:

# This line will copy some data off github
! git clone https://github.com/LenaRosendahl/lab7

Cloning into 'lab7'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7 (from 1)
Receiving objects: 100% (7/7), 104.34 KiB | 8.03 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
# These few lines will put you in the correct directory to access said data
!pwd
! ls
% cd /content/lab7
! pwd
!ls


/Users/younesstrittmatter/Documents/GitHub/younesStrittmatter/princeton/psyneulink-related/502B/book/content/Decision Making
Lab7_Decision_Making.ipynb lab7
intro.md                   notebooks


UsageError: Line magic function `%` not found.


In [ ]:
A = pd.read_pickle('Penalties.pkl')
B = pd.read_pickle('Coherence.pkl')

Ax = A.rt[A.condition == 'ZeroPenalty']
Ay = A.rt[A.condition == 'HighPenalty']
bins = np.linspace(0, 5, 20) # (start, stop, number of bins)
plt.hist([Ax, Ay], bins, alpha=0.5, label=['ZeroPenalty', 'HighPenalty'])
plt.title('RT Frequency by Penalty')
plt.legend(loc='upper right')
plt.show()

Bx = B.rt[B.condition == 'HighCoherence']
By = B.rt[B.condition == 'LowCoherence']
bins = np.linspace(0, 5, 20)
plt.hist([Bx, By], bins, alpha=0.5, label=['HighCoherence', 'LowCoherence'])
plt.title('RT Frequency by Coherence')
plt.legend(loc='upper right')
plt.show()

